In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 500
pd.options.display.max_columns = 500
pd.options.display.max_rows = 999

import re
from sklearn.externals import joblib

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()


from sklearn.model_selection import train_test_split


from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection  import KFold
from sklearn.model_selection  import StratifiedKFold
from sklearn.model_selection  import GridSearchCV

from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestClassifier


from sklearn.preprocessing import LabelEncoder
import xgboost

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve

from sklearn.metrics import confusion_matrix

import warnings; warnings.simplefilter('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

i_want_to_cv = False
i_want_to_publish = True

CORE_NUMBER = 4

import moose_modules
import os

DEPLOY_FOLDER = "deploy"
DATA_FOLDER = "data"
OUTPUT_FOLDER = "output"

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Цель работы
Автоматическая классификация обращений в техподдержку вместо используемого сейчас ручного труда.
#### Описание ситуации
* От пользователей поступает примерно 3000 обращений в месяц в техподдержку по различным каналам (телефон, почта, самостоятельная регистрация через ServiceDesk). Все эти обращения вручную классифицируются (зачем человек обратился) и маршрутизируются (кто будет делать).
* Недавно произошла смена программной платформы ServiceDesk с одного продукта (CA Service Desk) на другой (ServiceNow). Поэтому большая историческая база заявок у нас есть в одной системе, а новые заявки обрабатывать надо в другой.
* При смене платформы произошла полная смена классификатора. В старой было дерево предметных областей. В новой - дерево услуг, у каждой из которых есть категория, а у части - ещё подкатегория. При таком изменении подхода возникло несколько типов ситуаций:
    * По некоторым предметным областям работы в новой системе просто не ведутся.
    * У некоторых предметных областей из старой системы есть однозначное соответствие связки "услуга-категория-подкатегория" в новой.
    * У многих произошло разбиение на 2-3 категории. Например, был "Удалённый доступ через VPN". Он превратился в "Предоставление VPN", "Продление VPN", "Поддержка VPN". Смысл разбиения: предоставление надо согласовывать со Службой информационной безопасности, продление - нет, а поддержкой занимаются совсем другие люди.
    * Новый классификатор основан на услугах для пользователя, он ещё в процессе доработки, поэтому возникают ситуации дублирования. Например, когда пользователю надо установить клиент для VPN-подключения, это вроде бы "Установка прочего ПО". Но "Поддержка VPN" тоже подходит. Это сильно повлияет на оценку точности модели.
* При смене платформы также инженеры получили возможность переназначить заявку, не меняя значение классификатора. В старой системе такого не было. Это сильно повысило удобство работы в системе. Но, как следствие, мы получили много заявок с неправильным выбранным значением услуги. Очевидная идея переклассифицировать существующие заявки по последнему решавшему не работает по ряду причин (не будем на них останавливаться). Так что при обработке данных значительная часть работы заключается в корректной переклассификации заявок в новую систему услуг-категорий-подкатегорий.

#### Входные данные и постановка задачи
Входные данные - письмо пользователя с описанием проблемы. На первом этапе внедрения обрабатываются только входящие письма, т.е. у нас есть заголовок письма, тело письма и его автор. 

На выходе надо получить:
* Классификацию (услуга, категория, подкатегория).
* Признак срочности (надо ли эту заявку решать раньше прочих).
* Метрику уверенности в классификации.

Особенность постановки задачи в том, что допускается ответ классификатора "я не знаю". Более того, этот ответ лучше, чем ошибочная классификация. Следствия такой постановки:
* Можно игнорировать редкие классы, если модель на них будет выдавать ответ "я не знаю". Данные сильно несбалансированы (на этом остановимся позже), так что такая возможность сильно упрощает задачу.
* Требуется соблюсти некий баланс между количеством заявок, по которым модель даёт уверенный ответ и количеством ошибок при этом. 

#### Этапы процесса:
* Загрузить данные в датафреймы, склеить/переименовать столбцы
* Провести очистку и лемматизацию текстов
* Сгенерировать словарь токенов, которые не были распознаны лемматизатором. Список токенов отдельно проработать: что-то из него - это важный признак (названия ИС и подобное), а что-то - артефакты недостаточной очистки (например, "доба" - "добавочный телефон").
* Разбить заявки из snow на части для обучения и для валидации. Часть для обучения склеить с заявками из CA. Модель проверять только на валидационной части заявок из SNOW
* Обучить модель, учесть пороги, проверить метрики
* Склеить все заявки из SNOW и CA SD, Обучить модель на результате, отправить в продуктив.

Данные о заявках можно получать двумя основными способами:
* подключаться напрямую к БД и забирать данные оттуда
* сделать предварительную выгрузку в Excel и работать уже с ней.

У первого пути меньше лишних телодвижений, зато второй позволяет зафиксировать набор данных. Это важно в той ситуации, когда мы ищем оптимальный способ предварительной их трансформации перед обучением. 

In [2]:
#df_ca = pd.read_excel("data/raw_export_ca.xlsx")
#df_snow = pd.read_excel("data/raw_export_snow.xlsx")

df_ca = pd.read_excel(os.path.join(DATA_FOLDER, "raw_export_ca.xlsx"))
df_snow = pd.read_excel(os.path.join(DATA_FOLDER, "raw_export_snow.xlsx"))


df_snow.columns

Index(['Number', 'Short description', 'Description', 'Business service',
       'Category', 'Subcategory', 'Assignment group'],
      dtype='object')

Названия столбцов соответствуют названиям в интерфейсе ServiceNow. Но нам удобней работать с несколько иной структурой: 
* в первую очередь, надо склеить услугу, категорию и подкатегорию;
* у заявки есть 'Краткое описание' и 'Описание' (полное). Бывают такие ситуации, когда краткое представляет из себя первые N символов полного, а бывают такие, что в кратком описании содержится очень важная информация, которой нет в полном описании. поэтому их надо склеить, избежав дублирования
* столбцы, которые пока не участвуют в модели, удалим

In [3]:
def merge_summary_and_description(row):
    #print(row["summary"])
    
    if row["description"][:len(row["summary"])+1] == row["summary"]:
        return row["description"]
    else:
        return row["summary"] + " " + row["description"]


df_snow.rename(columns={"Номер":"ref_num", 
                        "Краткое описание":"summary", 
                        "Описание":"description", 
                        "Группа назначения":"AssignGroup",
                        
                        "Number":"ref_num", 
                        "Short description":"summary", 
                        "Description":"description", 
                        "Assignment group":"AssignGroup",
                        "Business service":"Бизнес-услуга", 
                        "Category":"Категория", 
                        "Subcategory":"Подкатегория"
                       
                       }, inplace = True)

df_ca["ref_num"] = df_ca["ref_num"].astype("str")
df_ca["summary"] = df_ca["summary"].astype("str")

df_snow["description"].fillna("",inplace=True)
df_ca["description"].fillna("",inplace=True)

df_snow["category"] = df_snow["Бизнес-услуга"].fillna("") + "@" + df_snow["Категория"].fillna("") + "@" + df_snow["Подкатегория"].fillna("")
df_snow.drop(['Бизнес-услуга', 'Категория', 'Подкатегория'], axis=1, inplace = True)

df_snow["raw_description"] = df_snow["summary"] + " " + df_snow["description"]
#df_snow["raw_description"] = df_snow.apply(merge_summary_and_description, axis=1)
df_ca["raw_description"] = df_ca.apply(merge_summary_and_description, axis=1)

df_snow["raw_description"].fillna(" ", inplace=True)
df_ca["raw_description"].fillna(" ", inplace=True)

In [4]:
print("Количество заявок из ServiceNow:", df_snow.shape[0])

Количество заявок из ServiceNow: 17645


Основные проблемы того, что мы получили в **raw_description**:
* огромные блоки "С уважением, бла-бла-бла"
* после которых идёт идёт абзац про коммерческую тайну в стиле "Если вы получили это письмо по ошибке - не надо было его читать"
* ServiceNow этот абзац при обработке почты разбивает переносами строки на несколько, так что вырезать регулярками 1 абзац, в котором есть выражение "коммерческая тайна" нельзя.
* вариантов этого текста около 10
* некоторые слова из него могут быть важными маркерами для категоризации, так что выкинуть все эти слова через stop-list тоже нельзя
* есть куча мусора в виде e-mail адресов, артефактов переписки типа "Re: Re> на: ha>> fw :" и т.д.
* номера телефонов нам тоже не нужны

Чем можно воспользоваться:
* Номера документов имеют фиксированный формат и иногда по номеру понятно, к какому разделу ERP документ относится. Заменим их по формату на токены, DOCBUH, DOCLOGISTIC. Очевидно, что для задачи классификации информация "в заявке указан номер бухгалтерского документа" полезнее, чем сам номер.
* IP и MAC адреса, также заменяем на фиксированные токены: IPADDRESS, MACADDRESS

In [5]:
cleaning_dict = {
    r"(\bстаф\b)|(\bестафф\b)": " estaff ",
    r"p:\\[\w]+": " networkdisk ",
    r" р\/с " : " расчетный счет ",
    r"( б\/н )|( безнал )" : " безналичный ",
    "кому: servicedesk": " ",
    "wi-fi": "wifi",

    ##############################################################
    ### много регулярок, местами включающих коммерческую тайну ###
    ##############################################################
    
    " скайп ": " skype ",
    "\t": " ",
    "\n": " "
}
    
# def remove_trash(x):
#     return re.sub(pattern="\|",repl=" ",string=x,count=0)
# 

def remove_multispaces(x):
    return re.sub(pattern=" +",repl=" ",string=str(x),count=0)


def remove_brackets(x):
    return re.sub(pattern="({)|(})",repl=" ",string=str(x),count=0)

def remove_questions(x):
    return re.sub(pattern="\?",repl="",string=str(x),count=0)

In [6]:
df_snow["text"] = df_snow["raw_description"].str.lower()
df_snow.replace(to_replace = {"text": cleaning_dict}, inplace = True, regex=True )
#df_snow["text2"] = df_snow["raw_description"].progress_apply(TEXT_PIPELINE)

In [7]:
df_ca["text"] = df_ca["raw_description"].str.lower()
df_ca.replace(to_replace = {"text": cleaning_dict}, inplace = True, regex=True )
#df_ca["text2"] = df_ca["raw_description"].progress_apply(TEXT_PIPELINE)

Сейчас в столбцах text у нас частично очищенное от мусора описание заявки со всем многообразием словоформ русского языка. Чтобы от этого многообразия избавиться, надо провести лемматизацию.

В качестве лемматизатора рассматривалось два варианта: *pymystem* и *pymorphy*.

*pymorphy* очень удобен в использовании, очень быстр (особенно с lru_cache) но недостаточно сильно снижает многообразие словоформ.

*pymystem* довольно жёсткий (лемматизирует то, что *pymorphy* уже считает леммой), быстрый в linux, но имеет очень большие накладные расходы (~0.7 секунды) на запуск под windows. Т.е. обрабатывать им слова по одному - дело долгое. Зато можно все тексты слить в один файл и его обработка пройдет довольно быстро. Кроме того, в качестве параметра можно подготовить fixlist, в котором заменять всякие сокращения и популярные опечатки. Важный минус - он просто игнорирует всю латиницу.

In [8]:
#df_ca["text"].to_csv("data/df_ca_text.csv", encoding="utf-8")
#df_snow["text"].to_csv("data/df_snow_text.csv", encoding="utf-8")

df_ca["text"].to_csv(os.path.join(DATA_FOLDER, "df_ca_text.csv"), encoding="utf-8")
df_snow["text"].to_csv(os.path.join(DATA_FOLDER, "df_snow_text.csv"), encoding="utf-8")



!"C:\Users\agurevich\.local\bin\mystem" -cdl --fixlist C:\Users\agurevich\python\moose\data\ts_fixlist.txt C:\Users\agurevich\python\moose\data\df_ca_text.csv C:\Users\agurevich\python\moose\data\df_ca_text.out
!"C:\Users\agurevich\.local\bin\mystem" -cdl --fixlist C:\Users\agurevich\python\moose\data\ts_fixlist.txt C:\Users\agurevich\python\moose\data\df_snow_text.csv C:\Users\agurevich\python\moose\data\df_snow_text.out

df_ca["text"] = pd.read_csv(os.path.join(DATA_FOLDER, "df_ca_text.out"), header=None)[1]
df_snow["text"] = pd.read_csv(os.path.join(DATA_FOLDER, "df_snow_text.out"), header=None)[1]

#df_ca["text"] = df_ca["text"].apply(only_letters)
#df_snow["text"] = df_snow["text"].apply(only_letters)
df_ca["text"] = df_ca["text"].apply(remove_brackets)
df_snow["text"] = df_snow["text"].apply(remove_brackets)

df_ca["text"] = df_ca["text"].apply(remove_multispaces)
df_snow["text"] = df_snow["text"].apply(remove_multispaces)            

В исходных данных много "грязных" данных с некорректной классификацией. В том числе заявок, классификация которых противоречит экспертному знанию. Функция moose_modules.reclassify_df применяет это знание.

Т.к. нам надо будет делать стратифицированое разбиение на обучающую и тестовую выборки, выбросим редкие заявки.

In [9]:
df_ca, df_snow = moose_modules.reclassify_df(df_ca, df_snow)

vc = pd.DataFrame(df_snow["category"].value_counts())
vc = vc[vc["category"]<4]
df_snow.drop(df_snow.index[df_snow["category"].isin(vc.index)], axis=0, inplace = True)    

vc = pd.DataFrame(df_ca["category"].value_counts())
vc = vc[vc["category"]<4]
df_ca.drop(df_ca.index[df_ca["category"].isin(vc.index)], axis=0, inplace = True)



df_snow.drop(df_snow.index[df_snow["category"] == "SAP@Служебные записки SAP@Выдача наличных средств"], inplace=True)
df_snow.drop(df_snow.index[df_snow["category"] == "Специфицирование@Доступ к специфицированию@"], inplace=True)
df_snow.drop(df_snow.index[df_snow["category"] == "ЦНСИ@Доступ к ЦНСИ@"], inplace=True)
df_snow.drop(df_snow.index[df_snow["category"] == "ТЕСС@Доступ к ТЕСС@"], inplace=True)
df_snow.drop(df_snow.index[df_snow["category"] == "ТЕСС@Поддержка ТЕСС@"], inplace=True)

In [10]:
strange_dict = {}
def collect_tokens(x):
    for i in x.split():
        if (i[-1]=="?"): strange_dict[i] = strange_dict.get(i, 0) + 1

df_ca["text"].apply(lambda x: collect_tokens(x))
df_snow["text"].apply(lambda x: collect_tokens(x));

In [11]:
strange_df =pd.DataFrame.from_dict(data =strange_dict, orient="index")
strange_df[0].fillna(" ", inplace = True)
strange_df["token"]= strange_df.index
strange_df["token"] = strange_df["token"].apply(remove_questions) 

In [12]:
stop_list = pd.read_csv("stopwords.csv", encoding="utf8",header=None)

for stopped in stop_list[0]:
    try:
        #print("try",stopped)
        strange_df.drop( strange_df[strange_df["token"]==stopped].index, inplace=True, axis=0)
    except:
        print("Not in strange_list:",stopped)

In [13]:
strange_df.drop(strange_df[strange_df["token"].str.len()<2].index, inplace = True)

In [14]:
strange_df.sort_values(0,ascending=False).to_excel(os.path.join(OUTPUT_FOLDER, "strange_df.xlsx"))

In [15]:
train_data, test_data = train_test_split(df_snow,train_size=.7, stratify=df_snow["category"], random_state=17)

train_data = pd.concat([df_ca, train_data], ignore_index=True)

train_data = train_data[train_data["category"].str.contains("@") ]
test_data = test_data[test_data["category"].str.contains("@") ]

train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [16]:
print(train_data.shape)
print(test_data.shape)

(53780, 7)
(5042, 7)


### Данные подготовлены, начнём обучение

In [17]:
Y_test = pd.DataFrame(test_data[["ref_num", "text", "raw_description","category"]])
target = train_data["category"]


In [18]:
vectorizer = TfidfVectorizer(min_df=2, max_df=1.0,  strip_accents=None,ngram_range = (1, 3), stop_words=list(stop_list))
train_vectorised_text = vectorizer.fit_transform(train_data["text"])
test_vectorised_text = vectorizer.transform(test_data["text"])

In [19]:
print("Объём словаря:",len(vectorizer.vocabulary_))

Объём словаря: 391111


In [20]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=241)

#### Обучение модели

In [32]:
def make_cv(model, cv_param_grid):
    gridCV = GridSearchCV(model, 
                          param_grid=cv_param_grid, 
                          scoring='f1_weighted', 
                          cv=kfold, 
                          n_jobs=CORE_NUMBER,
                          verbose=1)    
    t0 = time()
    gridCV.fit(train_vectorised_text, target)
    duration = time() - t0
    print("GridCV done in %fs " % (duration))
    return (gridCV.best_params_ ,gridCV.best_score_)

##### SGDClassifier

In [22]:
clf_sgd_bow = SGDClassifier() #class_weight=snow_dict
clf_sgd_bow_cv_params = {'alpha':[1e-05],
                          'penalty':['l2', None],
                          'loss': ['modified_huber'],
                          'learning_rate': ["constant"],
                          'class_weight' : [None],
                          'eta0' :[0.3,0.4,0.5],
                          'n_jobs':[CORE_NUMBER],
                        'random_state':17
                         }

clf_sgd_bow_train_params = {'alpha': 1e-05,
                             'class_weight': None,
                             'eta0': 0.1,
                             'learning_rate': 'constant',
                             'loss': 'modified_huber',
                             'penalty': None,
                             'n_jobs':CORE_NUMBER,
                             'random_state':17 }

if i_want_to_cv:
    cv_temp = make_cv(clf_sgd_bow, clf_sgd_bow_cv_params)
    print(cv_temp)
    clf_sgd_bow_train_params = cv_temp[0]
    

clf_sgd_bow.set_params(**clf_sgd_bow_train_params)
clf_sgd_bow.fit(train_vectorised_text, target)


sgd_f1_score = f1_score(Y_test ["category"],clf_sgd_bow.predict(test_vectorised_text), average="weighted", labels=clf_sgd_bow.classes_)
print("SGDClassifier F1 score", sgd_f1_score)

SGDClassifier F1 score 0.700764459966


In [23]:
t0 = time()
Y_test ["predicted_bow"] = clf_sgd_bow.predict(vectorizer.transform(Y_test["text"]))

duration = time() - t0
print("Prediction done in %fs " % (duration))

Prediction done in 1.120045s 


In [24]:
thresholds = pd.read_csv(os.path.join(DATA_FOLDER, "thresholds.csv"), encoding="utf-8", delimiter=";",index_col="category",decimal=",")
if (not ("f1_df" in locals())):
    f1_df = pd.DataFrame(Y_test["category"].value_counts())
    f1_df.rename(columns={"category":"count"},inplace=True)
    
    f1_df = pd.concat([f1_df, thresholds], axis=1, join_axes=[f1_df.index])
f1_df["threshold"] = thresholds["threshold"]
f1_df["metric_name"] = thresholds["metric_name"]
    

Если в выводе следующей ячейки что-то есть, то у нас появились новые категории, для которых не определены пороги.

In [25]:
f1_df[f1_df["metric_name"].isna()]

,count,threshold,metric_name


In [26]:

df_temp = pd.DataFrame(np.sort(clf_sgd_bow.predict_proba(vectorizer.transform(Y_test["text"])), axis=1))
Y_test["max_proba"] = df_temp[df_temp.shape[1]-1]
Y_test["second_proba"] = df_temp[df_temp.shape[1]-2]    
Y_test["delta"] = Y_test["max_proba"] -  Y_test["second_proba"]   

#Y_test2 =pd.concat([Y_test, pd.DataFrame([delta, max_proba, second_proba]).transpose().rename(columns={0:"delta", 1:"max_proba", 2:"second_proba"})], axis=1)
Y_test["new_metric"] = Y_test["max_proba"]/Y_test["second_proba"]
Y_test["log_new_metric"] = np.log(Y_test["max_proba"]/Y_test["second_proba"]) / np.log(Y_test["new_metric"][Y_test["new_metric"]!=float("inf")].max())
Y_test["bow_right"] = (Y_test["category"] == Y_test["predicted_bow"])

In [27]:
f1_df["bow_is_sure_count"]=0
f1_df["bow_is_right_count"]=0
Y_test["sure"]=False
for cat_name in f1_df.index:
    #print(cat_name)
    f1_df.loc[cat_name,"bow_is_sure_count"] = Y_test[(Y_test[f1_df.loc[cat_name, "metric_name"]]>f1_df.loc[cat_name, "threshold"])
                                            &(Y_test["predicted_bow"]==cat_name) ].shape[0]
    Y_test["sure"][(Y_test[f1_df.loc[cat_name, "metric_name"]]>f1_df.loc[cat_name, "threshold"])
                                            &(Y_test["predicted_bow"]==cat_name) ]=True
    f1_df.loc[cat_name,"bow_is_right_count"] = Y_test[(Y_test[f1_df.loc[cat_name, "metric_name"]]>f1_df.loc[cat_name, "threshold"])
                                            &(Y_test["predicted_bow"]==cat_name) 
                                            &(Y_test["predicted_bow"]==Y_test["category"])].shape[0]


In [28]:
print("Прогнозов:", f1_df["bow_is_sure_count"].sum())
print("Ошибок", f1_df["bow_is_sure_count"].sum()-f1_df["bow_is_right_count"].sum())
print("Процент прогнозов", f1_df["bow_is_sure_count"].sum()/f1_df["count"].sum())
print("Процент правильных", f1_df["bow_is_right_count"].sum()/f1_df["bow_is_sure_count"].sum())
print("|", f1_df["bow_is_sure_count"].sum(),
      "|", f1_df["bow_is_sure_count"].sum()-f1_df["bow_is_right_count"].sum(),
      "|", f1_df["bow_is_sure_count"].sum()/f1_df["count"].sum(),
      "|", f1_df["bow_is_right_count"].sum()/f1_df["bow_is_sure_count"].sum()
     )
f1_df["acc"] = f1_df["bow_is_right_count"]/f1_df["bow_is_sure_count"]


if ("f1_bow") in f1_df.columns: f1_df["f1_bow_prev"]=f1_df["f1_bow"]
f1_df["f1_bow"] = f1_score(Y_test ["category"],Y_test ["predicted_bow"], average=None, labels=f1_df.index)
if ("f1_bow_prev") in f1_df.columns: 
    f1_df["f1_bow_d"]=f1_df["f1_bow"]-f1_df["f1_bow_prev"] 
else: f1_df["f1_bow_d"]=0

# вывести счёт
#print("bow", str(f1_score(Y_test ["category"],Y_test ["predicted_bow"], average="weighted", labels=f1_df.index)))
#print("cfm", str(f1_score(Y_test ["category"],Y_test ["predicted_cfm"], average="weighted", labels=f1_df.index)))
#print("rf", str(f1_score(Y_test ["category"],Y_test ["predicted_rf"], average="weighted", labels=f1_df.index)))
#print("xgb", str(f1_score(Y_test ["category"],Y_test ["predicted_xgb"], average="weighted", labels=f1_df.index)))
#print("xgb bow", str(f1_score(Y_test ["category"],Y_test ["predicted_XGB_bow"], average="weighted", labels=f1_df["labels"])))





print("|{4:d}| {0:d} | {1:2d} | {2:0.3f} |{3:0.3f}|{5:0.5f}".format(f1_df["bow_is_sure_count"].sum()
                                              , f1_df["bow_is_sure_count"].sum()-f1_df["bow_is_right_count"].sum()
                                              , f1_df["bow_is_sure_count"].sum()/f1_df["count"].sum()
                                              , f1_df["bow_is_right_count"].sum()/f1_df["bow_is_sure_count"].sum()
                                              ,test_data.shape[0]
                                            ,sgd_f1_score
                                                           ))

Прогнозов: 3800
Ошибок 721
Процент прогнозов 0.753669178897
Процент правильных 0.810263157895
| 3800 | 721 | 0.753669178897 | 0.810263157895
|5042| 3800 | 721 | 0.754 |0.810|0.70076


Комментарий|Размер теста|Прогнозов|Ошибок|Процент прогнозов|Процент правильных|F1 модели
---|---|---|---|---|---
|4897| 3666 | 651 | 0.749 |0.822
удалил tел|4897| 3671 | 655 | 0.750 |0.822
замена на lotusnotes|4897| 3671 | 654 | 0.750 |0.822|0.712
.+ доб\..+|4897| 3663 | 650 | 0.748 |0.823|0.71372
.+ моб\..+|4897| 3680 | 657 | 0.751 |0.821|0.70894
пошаманил с регулярками и \b вместо \s|4897| 3681 | 656 | 0.752 |0.822|0.70931
|4897| 3688 | 663 | 0.753 |0.820|0.71014
|4897| 3695 | 664 | 0.755 |0.820|0.70989
oppa|4897| 3695 | 666 | 0.755 |0.820|0.70989
накинул данных|5042| 3800 | 721 | 0.754 |0.810|0.70076

In [30]:
cm = sns.light_palette("#2ecc71", as_cmap=True)



# Код для соблюдения NDA
if i_want_to_publish:
    f1_df.index=np.arange(0,f1_df.shape[0])
    s = f1_df[["count","threshold","metric_name","bow_is_sure_count","acc", "f1_bow","f1_bow_d"]].head().style.background_gradient(cmap=cm)
else:
    s = f1_df[["count","threshold","metric_name","bow_is_sure_count","acc", "f1_bow","f1_bow_d"]].style.background_gradient(cmap=cm)

s

,count,threshold,metric_name,bow_is_sure_count,acc,f1_bow,f1_bow_d
0,507,0.5,max_proba,491,0.881874,0.865116,0
1,501,0.55,max_proba,404,0.920792,0.814815,0
2,311,2.5,new_metric,309,0.860841,0.836941,0
3,208,1.5,new_metric,230,0.83913,0.865342,0
4,205,0.6,max_proba,111,0.918919,0.707447,0


In [31]:
!telegram-send "wake up"

#### Подготовка к установке
* объединим train и test, пересчитаем модель заново
* выложим в папку deploy всё то, что надо будет переносить во Flask

In [ ]:
merged = pd.concat([train_data, test_data], ignore_index=True)
merged_vectorised_text = vectorizer.fit_transform(merged["text"])
clf_sgd_bow.fit(merged_vectorised_text, merged["category"])

In [ ]:
joblib.dump(clf_sgd_bow, os.path.join(DEPLOY_FOLDER, "clf_sgd_snow.pkl")) 
joblib.dump(vectorizer, os.path.join(DEPLOY_FOLDER, "vec_snow.pkl")) 
joblib.dump(cleaning_dict, os.path.join(DEPLOY_FOLDER, "cleaning_dict.pkl")) 

thresholds.to_csv(os.path.join(DEPLOY_FOLDER, "thresholds.csv"), encoding="utf-8", sep=";",decimal=",", index=True)